In [1]:
from os import listdir
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from collections import Counter
import string
import multiprocessing
from tqdm import tqdm, trange
import operator
import pickle
lemmatizer = WordNetLemmatizer()

In [2]:
min_occ = 2

In [3]:
#files
trainDir = "aclimdb/train/"
testDir = "aclimdb/test/"
contenttrain = []
labeltrain = []
contenttest = []
labeltest = []
#extracting data
for i in listdir(trainDir+'pos'):
    contenttrain.append(open(trainDir+'pos/'+i,'r').read())
    labeltrain.append('1')
for i in listdir(trainDir+'neg'):
    contenttrain.append(open(trainDir+'neg/'+i,'r').read())
    labeltrain.append('0')
for i in listdir(testDir+'pos'):
    contenttest.append(open(testDir+'pos/'+i,'r').read())
    labeltest.append('1')
for i in listdir(testDir+'neg'):
    contenttest.append(open(testDir+'neg/'+i,'r').read())
    labeltest.append('0')

In [50]:
def vocab_sentences(sents):
    lexicon = []
    sentences = []
    for sent in tqdm(sents):
        
        refSent = ''.join(('' if i.isdigit() else i) for i in (sent) if (i not in string.punctuation) ).lower()
        allWords = word_tokenize(refSent)
        sentences.append(allWords)
        lexicon += list(allWords)
        
#     lexicon = [lemmatizer.lemmatize(i) for i in tqdm(lexicon)]
    w_counts = Counter(lexicon)
    lexicon2 =[]
    for w in w_counts:
        if w_counts[w]>min_occ:
            lexicon2.append(w)
    lexicon2.append('unk')
    lexicon2 = set(lexicon2)
    sentences = [[w if w in lexicon2 else 'unk' for w in s] for s in tqdm(sentences)]
    return sentences,lexicon2

In [12]:
#word to vec
def w2v(sentences):
    cores = multiprocessing.cpu_count()
    global model
    model = Word2Vec(sente, min_count=1, size=200, sg=1, iter=2, negative=10, workers=cores)
    model.save('./model/word2vec.model')

In [52]:
%time
#sentences and vocabulary
sente,vocab = vocab_sentences(contenttrain)

  0%|          | 50/25000 [00:00<00:54, 460.75it/s]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


100%|██████████| 25000/25000 [00:00<00:00, 26345.16it/s]


In [53]:
%time
#word to vec
w2v(sente)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 12.9 µs


In [54]:
# index to word dictionary
id_to_word = dict(enumerate(list(vocab)))
# word to index dictionary
word_to_id = {v: k for k, v in id_to_word.items()}

In [55]:
#sentence vector
sentVector = [[word_to_id[word] for word in line] for line in sente]
#maximum length vector
max_length = max([len(sent) for sent in sentVector])

In [56]:
# pad 0's to the end of each sequence
for i in trange(len(sentVector)):
    sentVector[i] = sentVector[i] + [0]*(max_length - len(sentVector[i]))

100%|██████████| 25000/25000 [00:00<00:00, 30523.28it/s]


In [59]:
# initalize an embeddings array which contains word vectors for all the words in vocabulary. These word vectors can 
# be accesed by the indices from this vector
embed = np.zeros((len(vocab)+1, 200))
for k,v in word_to_id.items():
    embed[v] = model[k]
# assigning 0's vector of size 200 for padding
# note: 0 index in vocabulary was assigned to empty word ''
embed[0] = np.zeros((200))
trainX = np.array(sentVector)
trainY = np.array(labeltrain)

In [63]:
np.savez('model/train_set.npz', trainX=trainX, trainY=trainY)
np.savez('model/embedding.npz', embed=embed)